# Задания по уроку 5.2

In [29]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomTreesEmbedding, VotingClassifier
from sklearn.linear_model import LogisticRegression

In [30]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Задачи с проверкой кода

3) Реализовать фукнцию для подсчета произвольной метрики из Scikit-Learn в режиме OOB. Подсказка: `oob_decision_function_`

In [31]:
# Write-up
def estimate_oob_metric(forest, metric):
    """
    Computes any classification metric in the out-of-bag mode.
        :arg forest: Scikit-Learn ensemble model
        :arg metric: callable with two arguments that returns a float
        
        :return oob_metric_value: float
    """
    return forest.oob_score_ 

In [32]:
forest = RandomForestClassifier(n_estimators=100, max_depth=8, criterion='entropy', oob_score=True, random_state=42)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
f1 = f1_score(y_train, y_pred)

estimate_oob_metric(forest, f1)

0.967032967032967

## Задачи с проверкой ответа

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.  

Случайный лес — стохастический алгоритм, а значит при каждом запуске может выдавать различный результат. В дальнейших задачах __необходимо__ выставить случайность: добавить параметр `random_state=8` как аргумент при создании ансамблей.

In [33]:
def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [34]:
forest = RandomForestClassifier(random_state=8)
compute_metric(forest)

0.965035

1) Напишите индекс самого важного признака по мнению случайного леса. Важности признаков указаны в переменной feature_importances_ случайного леса.

In [35]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
print('Важность признаков:', forest.feature_importances_)
max_value = max(forest.feature_importances_)
print('Максимальное значение:', max_value)
indx = list(forest.feature_importances_).index(max_value)
print('Индекс:', indx)

Важность признаков: [0.04870337 0.01359088 0.05326975 0.04755501 0.00728533 0.01394433
 0.06800084 0.10620999 0.00377029 0.00388577 0.02013892 0.00472399
 0.01130301 0.02240696 0.00427091 0.00525322 0.00938583 0.00351326
 0.00401842 0.00532146 0.07798688 0.02174901 0.06711483 0.15389236
 0.01064421 0.02026604 0.0318016  0.14466327 0.01012018 0.00521012]
Максимальное значение: 0.15389236463205394
Индекс: 23


2) Увеличьте количество в случайном лесе до 100. Как изменилось качество? Укажите его.

In [36]:
forest = RandomForestClassifier(random_state=8, n_estimators=100)
compute_metric(forest)

0.965035

3) Выставите количество деревьев в лесе равным 100 и включите режим подсчета out-of-bag score. Укажите полученное значение метрики accuracy, округленного до 6 знака после запятой.

In [37]:
def calc_accuracy(y_true, y_pred):
    P = np.sum(y_true == 1)
    N = np.sum(y_true == 0)
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))

    acc = (TP + TN) / (P + N)
    
    return acc, TP, TN, P, N

In [38]:
forest = RandomForestClassifier(random_state=8, n_estimators=100, oob_score=True)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
acc, TP, TN, P, N = calc_accuracy(y_train, y_pred)
print('TP = {}\nTN = {}\nP = {}\nN = {}\nacc = {}%'.format(TP, TN, P, N, acc * 100))

TP = 286
TN = 169
P = 286
N = 169
acc = 100.0%


4) Выключите режим random subspaces в случайном лесе. Укажите качество.

In [39]:
forest = RandomForestClassifier(n_estimators=100, oob_score=True)
compute_metric(forest)

0.972222

5) Выключите режим бутстрапа и переобучите случайный лес. Укажите качество.

In [40]:
forest = RandomForestClassifier(n_estimators=100, bootstrap=False)
compute_metric(forest)

0.965035

6) Какая средняя глубина у деревьев, которые получаются в случайном лесе со стандартными параметрами при его обучении на этой обучающей выборке?

In [41]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
sum_depth = 0
for i in range(forest.n_estimators):
    sum_depth = sum_depth + forest.estimators_[0].tree_.max_depth

print('Средняя глубина =', sum_depth/forest.n_estimators)

Средняя глубина = 7.0
